<a href="https://colab.research.google.com/github/devhyunjun/TIL/blob/master/pymongo_0531.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymongo
#드라이버

In [ ]:
!pip install geohash2

In [ ]:
#  code from team peer sungjun, PARK
import requests
import csv
import json
import uuid
import inspect
import re
import os
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/oneroom/items/{item_id}"
referer = "https://zigbang.com/home/oneroom/subways/414/items"
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/oneroom/subways/414/items',
    'Referer': 'https://www.zigbang.com/home/oneroom/map',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached
        
def export_items(items):
    fieldnames = [
        'id',
        'service_type',
        'address1',
        'address2',
        '_floor',
        'size',
        'size_m2',
        'sales_type',
        'rent',
        'deposit',
        'manage_cost',
        'manage_cost_inc',
        'elevator',
        'room_direction_text',
        'images',
        'parking',
        'near_subways',
        'options',
        'description',
        'title'
    ]
    filtered_item = []
    for item in items:
        item = item['item']
        item_id = item['id']
        url = magic_fstring_function(item_view_url)
        item['url'] = str(url)
        new_item = {}
        for fieldname in fieldnames:
            new_item[fieldname] = item[fieldname]
        filtered_item.append(new_item)
    return filtered_item
    
def describe_room_list(items):
    item_ids = list(items.keys())
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://www.zigbang.com/home/oneroom/map'
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
    return items

def get_room_list(
    items,
    deposit_gteq: int = 0,
    deposit_lteq: int = 8000,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wydjr",
    rent_gteq: int = 0,
    sales_type_in: str = "월세",
    service_type_eq: str = "원룸",
    ):
    resp = requests.get(
        url=item_list_api,
        params={
            "deposit_gteq": deposit_gteq,
            "deposit_lteq": deposit_lteq,
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
            "sales_type_in": sales_type_in,
            "service_type_eq": service_type_eq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()
    for section in new_items['sections']:
        for item in section['item_ids']:
            items[str(item)] = {}
    return items

def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['_floor'] or ''
    addr += ' '
    addr += item['title'] or ''
    return addr

def remove_non_digits(s):
    return re.sub(r'\D', '', s)
    
def crawl(geo_loc='wydr', apt_type='js'):
    if apt_type == 'ws':
        apt_type_kor = '월세'
    elif apt_type == 'js':
        apt_type_kor = '전세'
    items = {}
    items = get_room_list(items, geohash=geo_loc, sales_type_in=apt_type_kor)
    print(f'Crawling {geo_loc}({apt_type_kor})')
    describe_items = describe_room_list(items)
    define_column = export_items(describe_items)
    print(f'{geo_loc}의 {apt_type_kor} 매물 개수: {len(define_column)}')
    geo_dict = {}
    for item in define_column:
        full_addr = get_full_address(item)
        item['address'] = full_addr
        if full_addr not in geo_dict:
            geo_dict[full_addr] = []
        geo_dict[full_addr].append(item)
    item_list = []
    for apt in geo_dict.keys():
        item_list.append(geo_dict[apt][0])
    csv_name = f'{geo_loc}_{apt_type}.csv'
    with open(csv_name, 'w', encoding='utf-8', newline='') as csvfile:
        fieldnames = [
            'id',
            'service_type',
            'address1',
            'address2',
            '_floor',
            'size',
            'size_m2',
            'sales_type',
            'rent',
            'deposit',
            'manage_cost',
            'manage_cost_inc',
            'elevator',
            'room_direction_text',
            'images',
            'parking',
            'near_subways',
            'options',
            'description',
            'title'
        ]
        csvfile.write('\ufeff')
        writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
        writer.writeheader()
        # for item in item_list:
        #   print(item)
        return item_list

In [ ]:
crawl()

In [ ]:
import pymongo
client = pymongo.MongoClient("mongodb://13.208.64.53:27017")
client

MongoClient(host=['13.208.64.53:27017'], document_class=dict, tz_aware=False, connect=True)

In [ ]:
zigbang = client.crawling.zigbang

In [ ]:
items = crawl()
items

In [ ]:
zigbang.insert_many(items)

In [ ]:
QUERY = {
    "size" : {"$gte" : 7.0}
}

In [ ]:
result = zigbang.find(QUERY)

In [ ]:
import pandas as pd
pd.DataFrame(result).head()

,_id,id,service_type,address1,address2,_floor,size,size_m2,sales_type,rent,...,manage_cost_inc,elevator,room_direction_text,images,parking,near_subways,options,description,title,address
0,6476dfca770dc993f6e83612,36482996,원룸,경기도 양주시 고암동,597-2,2층,7.0,23.14,전세,0,...,"인터넷,TV",있음,남동향,"[{'index': 0, 'count': 1, 'url': 'https://ic.z...",가능,-,"에어컨,냉장고,세탁기,인덕션,신발장,싱크대","▶넓고 조용한 원룸\r\n▶1층에 편의점있는 편세권\r\n▶체육공원, 편의점, 세탁...",신축 저렴한 원룸 전세,경기도 양주시 고암동 597-2 2층 신축 저렴한 원룸 전세
1,6476dfca770dc993f6e83613,36497381,원룸,경기도 동두천시 지행동,667-3,3층,8.0,26.45,전세,0,...,"수도,인터넷,TV",없음,남향,"[{'index': 0, 'count': 1, 'url': 'https://ic.z...",가능,"지행역(1호선), 동두천중앙역(1호선)","에어컨,냉장고,세탁기,인덕션,책상,책장,침대,옷장,신발장,싱크대",동두천에서 대표 원룸전문 부동산 [[ AAA부동산 ]] 입니다. *^ㅇ^* \r\n...,"♥♥ 큰방 월세, 둘이 살아도 넓은원룸, 귀한 전세","경기도 동두천시 지행동 667-3 3층 ♥♥ 큰방 월세, 둘이 살아도 넓은원룸, 귀..."
2,6476dfca770dc993f6e83614,36324519,원룸,경기도 포천시 소흘읍 송우리,728-20,1층,8.0,26.45,전세,0,...,"수도,인터넷,TV",없음,남서향,"[{'index': 0, 'count': 1, 'url': 'https://ic.z...",가능,-,"에어컨,냉장고,세탁기,인덕션,옷장,신발장,싱크대",주차좋은 조용한 원룸입니다 차 있으신 분 권합니다 \r\n\r\n깨끗하고 조용합니...,신축급 넘 깨끗하고 조용한 넓은 분리형 원룸,경기도 포천시 소흘읍 송우리 728-20 1층 신축급 넘 깨끗하고 조용한 넓은 분리...
3,6476dfca770dc993f6e83615,36767029,원룸,경기도 동두천시 지행동,675-4,2층,7.0,23.14,전세,0,...,"수도,인터넷,TV",없음,남서향,"[{'index': 0, 'count': 1, 'url': 'https://ic.z...",가능,"지행역(1호선), 동두천중앙역(1호선)","에어컨,냉장고,세탁기,인덕션,전자레인지,책상,책장,침대,옷장,신발장,싱크대",☝ 위치 및 특징\r\n- 원룸촌에 위치한 조용한 원룸입니다.\r\n- 주방분리형이...,😊청소완료😊주방분리형 완전 넓은1.5룸 전세입니다,경기도 동두천시 지행동 675-4 2층 😊청소완료😊주방분리형 완전 넓은1.5룸 전세입니다
4,6476dfca770dc993f6e83616,36775621,원룸,경기도 포천시 소흘읍 송우리,726-10,3층,8.0,26.45,전세,0,...,-,없음,북서향,"[{'index': 0, 'count': 1, 'url': 'https://ic.z...",불가능,-,"에어컨,냉장고,세탁기,가스레인지,책장,신발장,싱크대",📸100%실사진만 보여드립니다.\r\n\r\n💜 소재지 : 소흘읍 송우리 먹자골목 ...,송우리 3단지 인근 넓고 쾌적한 원룸 전세,경기도 포천시 소흘읍 송우리 726-10 3층 송우리 3단지 인근 넓고 쾌적한 원룸 전세
